In [ ]:
import numpy as np

class DecisionTreeC45:
    def __init__(self, max_depth=None, min_samples_split=2, min_gain=1e-4):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_gain = min_gain
        
    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.tree_ = self._grow_tree(X, y)
        
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        if (depth >= self.max_depth if self.max_depth else False) or n_labels == 1 or n_samples < self.min_samples_split:
            return LeafNode(y)
        
        gain = np.zeros(n_features)
        for feature in range(n_features):
            gain[feature] = self._information_gain(X[:, feature], y)
        
        best_feature = np.argmax(gain)
        if gain[best_feature] < self.min_gain:
            return LeafNode(y)
        
        mask = X[:, best_feature] == 1
        X_true, y_true = X[mask], y[mask]
        X_false, y_false = X[~mask], y[~mask]
        
        true_branch = self._grow_tree(X_true, y_true, depth + 1)
        false_branch = self._grow_tree(X_false, y_false, depth + 1)
        
        return DecisionNode(best_feature, true_branch, false_branch)
    
    def _information_gain(self, feature, y):
        n = len(y)
        parent_entropy = self._entropy(y)
        
        if n == 0:
            return 0
        
        true_index = np.where(feature == 1)[0]
        false_index = np.where(feature == 0)[0]
        
        n_true, n_false = len(true_index), len(false_index)
        true_entropy = self._entropy(y[true_index])
        false_entropy = self._entropy(y[false_index])
        
        child_entropy = (n_true / n) * true_entropy + (n_false / n) * false_entropy
        
        return parent_entropy - child_entropy
    
    def _entropy(self, y):
        n = len(y)
        if n == 0:
            return 0
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / n
        return -np.sum(probabilities * np.log2(probabilities))
    
    def predict(self, X):
        return np.array([self._predict(x, self.tree_) for x in X])
    
    def _predict(self, x, node):
        if isinstance(node, LeafNode):
            return node.predicted_class
        if x[node.feature]:
            return self._predict(x, node.true_branch)
        else:
            return self._predict(x, node.false_branch)
        
class LeafNode:
    def __init__(self, y):
        self.y = y
        self.predicted_class = np.argmax(np.bincount(y))
        
class DecisionNode:
    def __init__(self, feature, true_branch, false_branch):
        self.feature = feature
        self.true_branch = true_branch
        self.false_branch = false_branch
